In [12]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os
import json
from copy import deepcopy

In [13]:
api_key=os.getenv("GROQ_API_KEY")
model_name=os.getenv("MODEL_NAME")
print(api_key)
print(model_name)

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
llama3-8b-8192


In [18]:
client=ChatGroq(api_key=api_key,model=model_name)

In [43]:
def get_response(messages):
        messages=deepcopy(messages)

        system_prompt="""
            You are a helpful AI assistant for a coffee shop application which serves drinks and pastries.
            Your task is to determine whether the user is asking something relevant to the coffee shop or not.
            The user is allowed to:
            1. Ask questions about the coffee shop, like location, working hours, menue items and coffee shop related questions.
            2. Ask questions about menue items, they can ask for ingredients in an item and more details about the item.
            3. Make an order.
            4. ASk about recommendations of what to buy.

            The user is NOT allowed to:
            1. Ask questions about anything else other than our coffee shop.
            2. Ask questions about the staff or how to make a certain menue item.

            Your output must be in a valid JSON format.
            Each key and value must be wrapped in double quotes.
            Do not include any extra text or explanation outside the JSON.
            Strictly return only the JSON object.
            {
            "chain of thought": go over each of the points above and make see if the message lies under this point or not. Then you write some your thoughts about what point is this input relevant to.
            "decision": "allowed" or "not allowed". Pick one of those. and only write the word.
            "message": leave the message empty if it's allowed, otherwise write "Sorry, I can't help with that. Can I help you with your order?"
            }
            """

        input_messages=[{"role":"system","content":system_prompt}]+messages[-3:]
        chatbot_output=get_chatbot_response(client,input_messages,0.7)
        print(chatbot_output)
        output = postprocess(chatbot_output)
        
        return output
        
        

In [36]:
def postprocess(output):
    try:
        # Remove triple backticks or markdown code block if present
        cleaned_output = output.strip().strip("`").strip()
        parsed_output = json.loads(cleaned_output)
    except Exception as e:
        raise ValueError(f"Error parsing model output: {output}") from e

    return {
        "role": "assistant",
        "content": parsed_output['message'],
        "memory": {
            "agent": "guard_agent",
            "guard_decision": parsed_output['decision']
        }
    }


In [44]:
def get_chatbot_response(client,messages,temperature=0):
    input_messages=[]
    for message in messages:
        input_messages.append({"role":message["role"],"content":message["content"]})

        response=client.invoke(input=input_messages,temperature=temperature)

    return response.content


In [45]:
test_messages = [
    {"role": "user", "content": "Please tell us about the staff?"}
]

response = get_response(test_messages)
print(response)


{
"chain of thought": ["staff", "coffee shop"],
"decision": "not allowed",
"message": "Sorry, I can't help with that. Can I help you with your order?"
}
{'role': 'assistant', 'content': "Sorry, I can't help with that. Can I help you with your order?", 'memory': {'agent': 'guard_agent', 'guard_decision': 'not allowed'}}
